<h1> Dataset

In [1]:
import numpy as np
import pandas as pd

manuale = pd.read_csv('data/manuale.csv', sep=';')
manuale


,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15,98,86,high risk
1,35,140,90,13,98,70,high risk
2,29,90,70,8,100,80,high risk
3,30,140,85,7,98,70,high risk
4,35,120,60,61,98,76,low risk
5,23,140,80,701,98,70,high risk
6,23,130,70,701,98,78,mid risk
7,35,85,60,11,102,86,high risk
8,32,120,90,69,98,70,mid risk
9,42,130,80,18,98,70,high risk


Con il comando describe visualizziamo una sintesi statistica del dataset

In [2]:
manuale.describe()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,28.600000,116.166667,76.466667,198.333333,98.366667,76.000000
std,11.000313,20.158139,12.637474,283.249778,0.999425,6.843068
min,10.000000,70.000000,50.000000,7.000000,98.000000,66.000000
25%,21.000000,100.000000,66.250000,15.750000,98.000000,70.000000
50%,25.000000,120.000000,80.000000,70.500000,98.000000,76.000000
75%,35.000000,130.000000,83.750000,75.000000,98.000000,80.000000
max,50.000000,140.000000,100.000000,701.000000,102.000000,90.000000


Consideriamo anche il dataset ripulito dai valori anomali

In [3]:
# Calcoliamo la mediana dei valori non anomali
manuale_c = pd.read_csv('data/manuale.csv', sep=';')
# Calcolo della mediana dei valori NON anomali (40 <= BS <= 200)
bs_median = manuale_c.loc[(manuale_c['BS'] >= 40) & (manuale_c['BS'] <= 200), 'BS'].median()
print(bs_median)
# Sostituzione dei valori anomali con la mediana calcolata
# Sostituzione condizionale
manuale_c.loc[(manuale_c['BS'] > 200) | (manuale_c['BS'] < 40), 'BS'] = bs_median
manuale_c   #dataset manuale.csv con la sostitituzione dei valori anomali, cioè ripulito




72.0


,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,72,98,86,high risk
1,35,140,90,72,98,70,high risk
2,29,90,70,72,100,80,high risk
3,30,140,85,72,98,70,high risk
4,35,120,60,61,98,76,low risk
5,23,140,80,72,98,70,high risk
6,23,130,70,72,98,78,mid risk
7,35,85,60,72,102,86,high risk
8,32,120,90,69,98,70,mid risk
9,42,130,80,72,98,70,high risk


Implementiamo i seguenti modelli manualmente:
<span style="color:red"> KNN , Gaussian Naive Bayes</span> 

<h1> K-Nearest-Neighbors

La formula della distanza euclidea tra due punti in uno spazio n-dimensionale è:

$$
d(p_1, p_2) = \sqrt{\sum_{i=1}^{n} (p_{1,i} - p_{2,i})^2}
$$

Per uno spazio bidimensionale, la formula diventa:

$$
d(p_1, p_2) = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}
$$

In [4]:
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1) - 1):  # Escludere l'ultima colonna (la classe target)
        distance += (row1[i] - row2[i]) ** 2
    return np.sqrt(distance)

La funzione knn_predict implementa l’algoritmo KNN. Dato un set di punti di training  X , le rispettive etichette  y , un set di punti di test e  k , restituisce le predizioni.

In [5]:
from collections import Counter

def knn_predict(X_train, y_train, X_test, k):
    predictions = []
    for test_point in X_test:
        # Calcola la distanza tra il punto di test e tutti i punti di training
        distances = [(euclidean_distance(test_point, x_train), y_train[i]) for i, x_train in enumerate(X_train)]
        # Ordina le distanze in ordine crescente
        distances.sort(key=lambda x: x[0])
        # Seleziona i primi k vicini
        k_nearest = [dist[1] for dist in distances[:k]]
        # Predici la classe in base alla votazione a maggioranza
        most_common = Counter(k_nearest).most_common(1)[0][0]
        predictions.append(most_common)
    return predictions

Cross-validation con k-fold : divide il dataset in  k -fold, usa ciascun fold come test set una volta, mentre il resto dei fold viene usato come training set. Successivamente calcoliamo l’accuratezza media su tutti i fold.

In [6]:
# Funzione per suddividere i dati in k-fold
def k_fold_split(X, y, k):
    fold_size = len(X) // k
    X_folds = []
    y_folds = []
    for i in range(k):
        X_folds.append(X[i * fold_size: (i + 1) * fold_size])
        y_folds.append(y[i * fold_size: (i + 1) * fold_size])
    return X_folds, y_folds

def cross_validate_knn(X, y, k_neighbors, num_folds=5):
    X_folds, y_folds = k_fold_split(X, y, num_folds)
    
    accuracies = []  # Per memorizzare le accuratezze per ciascun fold
    for i in range(num_folds):
        # Fold i-esimo come test set
        X_test_fold = X_folds[i]
        y_test_fold = y_folds[i]
        
        # Tutti gli altri fold come training set
        X_train_folds = np.concatenate([X_folds[j] for j in range(num_folds) if j != i])
        y_train_folds = np.concatenate([y_folds[j] for j in range(num_folds) if j != i])
        
        # Previsione con KNN
        y_pred = knn_predict(X_train_folds, y_train_folds, X_test_fold, k_neighbors)
        
        # Accuratezza per questo fold
        accuracy = np.mean(np.array(y_pred) == np.array(y_test_fold))
        accuracies.append(accuracy)
    
    # Accuratezza media sui fold
    return np.mean(accuracies)


Applichiamo il classificatore knn progettato senza set di holdout, cioè sullo stesso set di dati con cui addestriamo il classificatore (il training set e il test set sono gli stessi)

In [7]:

# Separiamo le feature (X) e le etichette (y)
X = manuale.iloc[:, :-1].values  # Tutte le colonne tranne l'ultima (feature)
y = manuale.iloc[:, -1].values    # L'ultima colonna (target)

y_final_pred = knn_predict(X, y, X, 1)

# Calcoliamo e stampiamo l'accuratezza finale usando lo stesso dataset
final_accuracy = np.mean(y_final_pred == y)
print(f'Accuratezza finale sui dati : {final_accuracy:.4f}')

Accuratezza finale sui dati : 1.0000


Notiamo come con la scelta di  k = 1  si ottenga un’accuratezza del 100% sui dati di test.. Infatti con  k = 1 , il modello classifica ogni punto basandosi esclusivamente sul suo vicino più vicino. Se i dati di test sono molto simili ai dati di addestramento o addirittura sovrapposti,  k = 1  può memorizzare esattamente il dataset e restituire sempre il risultato corretto (overfitting). Valori di  k  più alti (ma non troppo alti) rendono il modello meno sensibile al rumore, perché considerano più punti per fare una classificazione. 

Utilizziamo una funzione per trovare il miglior  k (numero dei vicini) da utilizzare nel knn.  Testiamo diversi valori di  k  e valutiamo le prestazioni medie tramite cross-validation

In [8]:
def find_best_k(X, y, k_values, num_folds=5):
    best_k = k_values[0]
    best_accuracy = 0
    
    for k in k_values:
        # Eseguiamo la cross-validation per ciascun k
        accuracy = cross_validate_knn(X, y, k, num_folds)
        print(f'Accuracy for k={k}: {accuracy:.4f}')
        
        # Se l'accuratezza per questo k è migliore, aggiorna il miglior k
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_k = k
    
    print(f'Miglior valore di k: {best_k} con un\'accuratezza di {best_accuracy:.4f}')
    return best_k

In [9]:

# Separiamo le feature (X) e le etichette (y)
X = manuale.iloc[:, :-1].values  # Tutte le colonne tranne l'ultima (feature)
y = manuale.iloc[:, -1].values    # L'ultima colonna (target)
# Eseguiamo la ricerca del miglior valore di k
k_values = list(range(1, 11))  # Prova k da 1 a 10
best_k = find_best_k(X, y, k_values, num_folds=5)  # Cross-validation a 5 fold

# Valutazione finale: Uso del miglior k trovato per fare predizioni su tutto il dataset
y_final_pred = knn_predict(X, y, X, best_k)

# Accuratezza finale usando lo stesso dataset
final_accuracy = np.mean(y_final_pred == y)
print(f'Accuratezza finale sui dati (con k={best_k}): {final_accuracy:.4f}')


Accuracy for k=1: 0.5667
Accuracy for k=2: 0.5667
Accuracy for k=3: 0.5667
Accuracy for k=4: 0.5667
Accuracy for k=5: 0.6333
Accuracy for k=6: 0.6000
Accuracy for k=7: 0.6333
Accuracy for k=8: 0.6000
Accuracy for k=9: 0.5667
Accuracy for k=10: 0.5333
Miglior valore di k: 5 con un'accuratezza di 0.6333
Accuratezza finale sui dati (con k=5): 0.8333


Proviamo ad usare il dataset manuale_c, ovvero quello ripulito

In [10]:
# Separiamo le feature (Xp) e le etichette (yp) però sul dataset ripulito 
Xp = manuale_c.iloc[:, :-1].values  # Tutte le colonne tranne l'ultima (feature)
yp = manuale_c.iloc[:, -1].values    # L'ultima colonna (target)
#Ricerca del miglior valore di k
k_values = list(range(1, 11))  # Prova k da 1 a 10
best_k = find_best_k(Xp, yp, k_values, num_folds=5)  # Cross-validation a 5 fold

# Valutazione finale: Uso del miglior k trovato per fare predizioni su tutto il dataset
y_final_pred = knn_predict(Xp, yp, Xp, best_k)


final_accuracy = np.mean(y_final_pred == yp)
print(f'Accuratezza finale sui dati (con k={best_k}): {final_accuracy:.4f}')


Accuracy for k=1: 0.4333
Accuracy for k=2: 0.4333
Accuracy for k=3: 0.4667
Accuracy for k=4: 0.5000
Accuracy for k=5: 0.5000
Accuracy for k=6: 0.5000
Accuracy for k=7: 0.5000
Accuracy for k=8: 0.5000
Accuracy for k=9: 0.5000
Accuracy for k=10: 0.4333
Miglior valore di k: 4 con un'accuratezza di 0.5000
Accuratezza finale sui dati (con k=4): 0.8333


<h1> Gaussian Naive Bayes

Definiamo le funzioni che serviranno per implementare il classificatore GNB

In [11]:
def calcola_parametri(X, y):
    # Identifichiamo le classi uniche
    classes = np.unique(y)
    
    # Dizionario per contenere i parametri per ogni classe
    parameters = {}
    
    # Calcola media, varianza e probabilità a priori per ogni classe
    for cls in classes:
        X_c = X[y == cls]  # Dati appartenenti alla classe corrente
        parameters[cls] = {
            'mean': X_c.mean(axis=0),  # Media di ciascuna feature
            'var': X_c.var(axis=0),    # Varianza di ciascuna feature
            'prior': X_c.shape[0] / X.shape[0]  # Probabilità a priori
        }
    
    return parameters

In [12]:
def gaussian_probability(x, mean, var):
    # Aumenta epsilon per evitare varianze troppo piccole che causano problemi numerici
    epsilon = 1e-1 # Valore piccolo ma più grande di quello precedente
    coefficient = 1 / np.sqrt(2 * np.pi * (var + epsilon))
    exponent = np.exp(-((x - mean) ** 2) / (2 * (var + epsilon)))
    return coefficient * exponent

In [13]:
def class_probability(x, parameters):
    probabilities = {}
    
    # Probabilità per ciascuna classe
    for cls, params in parameters.items():
        prior = np.log(params['prior'])  # Usa log per la probabilità a priori
        likelihood = np.sum(np.log(gaussian_probability(x, params['mean'], params['var'])))
        probabilities[cls] = prior + likelihood
    
    return probabilities


In [14]:
def predict(X, parameters):
    predictions = []
    for x in X: # Per ogni esempio nei dati di input
        class_probabilities = class_probability(x, parameters)
        # Seleziona la classe con la probabilità a posteriori massima
        predictions.append(max(class_probabilities, key=class_probabilities.get))
    
    return np.array(predictions)


In [15]:
def calculate_accuracy(y_true, y_pred):
    correct_predictions = np.sum(y_true == y_pred)  # Conta le predizioni corrette
    accuracy = correct_predictions / len(y_true)  # Divide per il numero totale di esempi
    return accuracy

Proviamo con il dataset non ripulito

In [ ]:
# Calcolo dei parametri (media, varianza, probabilità a priori) per ogni classe 
parameters = calcola_parametri(X, y)

X_test = X  # Utilizzo dati esistenti

# Previsione  delle classi
predictions = predict(X_test, parameters)

Predizioni: ['high risk' 'high risk' 'high risk' 'high risk' 'low risk' 'low risk'
 'low risk' 'high risk' 'low risk' 'mid risk' 'low risk' 'low risk'
 'low risk' 'mid risk' 'high risk' 'low risk' 'high risk' 'high risk'
 'mid risk' 'low risk' 'high risk' 'mid risk' 'low risk' 'low risk'
 'low risk' 'low risk' 'low risk' 'low risk' 'mid risk' 'low risk']
Etichette vere: ['high risk' 'high risk' 'high risk' 'high risk' 'low risk' 'high risk'
 'mid risk' 'high risk' 'mid risk' 'high risk' 'low risk' 'mid risk'
 'low risk' 'mid risk' 'mid risk' 'low risk' 'high risk' 'high risk'
 'mid risk' 'low risk' 'high risk' 'mid risk' 'low risk' 'low risk'
 'low risk' 'low risk' 'low risk' 'low risk' 'low risk' 'low risk']


In [17]:

predictions = predict(X, parameters)  # Prevediamo con il modello Gaussian Naive Bayes

# Valutazione dell'accuratezza
accuracy = calculate_accuracy(y, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.77


Proviamo con il dataset ripulito

In [18]:
parameters_p = calcola_parametri(Xp, yp)
predictions_p = predict(Xp, parameters_p)  # Prevediamo con il modello Gaussian Naive Bayes

# Valutazione dell'accuratezza
accuracy = calculate_accuracy(yp, predictions_p)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.83


Definiamo un algoritmo per la matrice di confusione

In [19]:
def confusion_matrix(y_true, y_pred):
    # Identifica le classi uniche
    classes = np.unique(np.concatenate((y_true, y_pred)))
    # Crea una matrice di confusione vuota
    matrix = np.zeros((len(classes), len(classes)), dtype=int)
    
    # Popola la matrice
    for true, pred in zip(y_true, y_pred):
        true_index = np.where(classes == true)[0][0]  # Trova l'indice della classe vera
        pred_index = np.where(classes == pred)[0][0]  # Trova l'indice della classe prevista
        matrix[true_index, pred_index] += 1
    
    return matrix, classes




Matrice di confusione con l'utilizzo di GNB sul dataset non ripulito

In [20]:
# Usiamo la funzione per calcolare la matrice di confusione
cm, classes = confusion_matrix(y, predictions)

# Stampiamo la matrice di confusione
print("Matrice di confusione:")
print(cm)



# Stampa le etichette delle classi
print("Etichette delle classi:", classes)

Matrice di confusione:
[[ 8  1  1]
 [ 0 12  1]
 [ 1  3  3]]
Etichette delle classi: ['high risk' 'low risk' 'mid risk']


Matrice di confusione con l'utilizzo di GNB sul dataset ripulito

In [21]:
cm_p, classes = confusion_matrix(yp, predictions_p)
print("Matrice di confusione:")
print(cm_p)

# Stampiamo le etichette delle classi
print("Etichette delle classi:", classes)

Matrice di confusione:
[[10  0  0]
 [ 2 11  0]
 [ 2  1  4]]
Etichette delle classi: ['high risk' 'low risk' 'mid risk']
